# Размещение баннеров

Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare. Часть открытых данных есть, например, на сайте archive.org: https://archive.org/details/201309_foursquare_dataset_umn Скачаем любым удобным образом архив fsq.zip с этой страницы. Нас будет интересовать файл checkins.dat. Открыв его, увидим следующую структуру:

id | user_id | venue_id | latitude | longitude | created_at

---------+---------+----------+-------------------+-------------------+---------------------

984301 | 2041916 | 5222 | | | 2012-04-21 17:39:01

984222 | 15824 | 5222 | 38.8951118 | -77.0363658 | 2012-04-21 17:43:47

984315 | 1764391 | 5222 | | | 2012-04-21 17:37:18

984234 | 44652 | 5222 | 33.800745 | -84.41052 | 2012-04-21 17:43:43

...

Для удобной работы с этим документом преобразуем его к формату csv, удалив строки, не содержащие координат — они неинформативны для нас:

id,user_id,venue_id,latitude,longitude,created_at

984222,15824,5222,38.8951118,-77.0363658,2012-04-21T17:43:47

984234,44652,5222,33.800745,-84.41052,2012-04-21T17:43:43

984291,105054,5222,45.5234515,-122.6762071,2012-04-21T17:39:22

...

### 1

С помощью pandas построим DataFrame и убедимся, что все 396634 строки с координатами считаны успешно.

In [1]:
import numpy as np
import pandas as pd

from sklearn import cluster

Читаем датасет.

In [2]:
df = pd.read_csv('checkins.dat', delimiter='|', skipinitialspace=True)
df.head(10)

c:\users\asus\appdata\local\programs\python\python36-32\lib\site-packages\IPython\core\interactiveshell.py:3018: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,user_id,venue_id,latitude,longitude,created_at
0,---------+---------+----------+---------------...,NaN,NaN,NaN,NaN,NaN
1,984301,2041916.0,5222.0,NaN,NaN,2012-04-21 17:39:01
2,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-21 17:43:47
3,984315,1764391.0,5222.0,NaN,NaN,2012-04-21 17:37:18
4,984234,44652.0,5222.0,33.800745,-84.410520,2012-04-21 17:43:43
5,984249,2146840.0,5222.0,NaN,NaN,2012-04-21 17:42:58
6,984268,2146843.0,5222.0,NaN,NaN,2012-04-21 17:42:38
7,984281,2146846.0,5222.0,NaN,NaN,2012-04-21 17:39:40
8,984291,105054.0,5222.0,45.523452,-122.676207,2012-04-21 17:39:22
9,6651,1338710.0,219703.0,NaN,NaN,2011-12-08 23:11:23


In [3]:
df.shape

(1021968, 6)

In [4]:
df.columns = [c.strip() for c in df.columns]
df.columns

Index(['id', 'user_id', 'venue_id', 'latitude', 'longitude', 'created_at'], dtype='object')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1021968 entries, 0 to 1021967
Data columns (total 6 columns):
id            1021968 non-null object
user_id       1021966 non-null float64
venue_id      1021966 non-null float64
latitude      396634 non-null float64
longitude     396634 non-null float64
created_at    1021966 non-null object
dtypes: float64(4), object(2)
memory usage: 39.0+ MB


Удаляем строки, не содержащие координат.

In [6]:
df.dropna(subset=['latitude', 'longitude'], axis=0, inplace=True)
df.shape

(396634, 6)

In [7]:
df.reset_index(drop=True, inplace=True)
df.head()

,id,user_id,venue_id,latitude,longitude,created_at
0,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-21 17:43:47
1,984234,44652.0,5222.0,33.800745,-84.410520,2012-04-21 17:43:43
2,984291,105054.0,5222.0,45.523452,-122.676207,2012-04-21 17:39:22
3,984318,2146539.0,5222.0,40.764462,-111.904565,2012-04-21 17:35:46
4,984232,93870.0,380645.0,33.448377,-112.074037,2012-04-21 17:38:18


### 2

Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача — хороший повод познакомиться с алгоритмом MeanShift, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в sklearn user guide, а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

_Примечание: на 396634 строках кластеризация будет работать долго. Быть очень терпеливым не возбраняется — результат от этого только улучшится. Но для того, чтобы сдать задание, понадобится сабсет из первых 100 тысяч строк. Это компромисс между качеством и затраченным временем. Обучение алгоритма на всём датасете занимает около часа, а на 100 тыс. строк — примерно 2 минуты, однако этого достаточно для получения корректных результатов._

In [8]:
data = df.loc[:99999, ['latitude', 'longitude']]
data.shape

(100000, 2)

In [9]:
ms_cluster = cluster.MeanShift(bandwidth=0.1)
ms_cluster.fit(data)

MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=1,
     n_jobs=None, seeds=None)

Количество полученных кластеров

In [10]:
len(ms_cluster.cluster_centers_)

3231

Для каждой точки создадим дополнительный признак в виде метки кластера.

In [11]:
data['cluster'] = ms_cluster.labels_

Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

In [12]:
clusters15 = []
for i in range(len(ms_cluster.cluster_centers_)):
    if data['cluster'][data['cluster']==i].count() > 15:
        clusters15.append(i)
clusters15[-10:]

[595, 615, 636, 657, 672, 684, 727, 884, 1343, 1350]

In [13]:
centers15 = [ms_cluster.cluster_centers_[i] for i in clusters15]
centers15[:2]

[array([ 40.7177164 , -73.99183542]), array([  33.44943805, -112.00213969])]

При желании увидеть получившиеся результаты на карте можно передать центры получившихся кластеров в один из инструментов визуализации. Например, сайт [mapcustomizer.com](https://www.mapcustomizer.com/) имеет функцию Bulk Entry, куда можно вставить центры полученных кластеров в формате:

In [14]:
s = ''
for c in centers15:
    s += '{0} {1}\n'.format(c[0], c[1])
with open('for_map.txt', 'w') as fout:
    fout.write(s)

![alt text](map.JPG "Центры кластеров на карте")

Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу Carnival Cruise Line адреса всех офисов:  
- 33.751277, -118.188740 (Los Angeles)
- 25.867736, -80.324116 (Miami)
- 51.503016, -0.075479 (London)
- 52.378894, 4.885084 (Amsterdam)
- 39.366487, 117.036146 (Beijing)
- -33.868457, 151.205134 (Sydney)

Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.   
_Примечание: при подсчете расстояний и в кластеризации можно пренебречь тем, что Земля круглая, так как в точках, расположенных близко друг к другу погрешность мала, а в остальных точках значение достаточно велико._  
Для сдачи задания выберите из получившихся 20 центров тот, который наименее удален от ближайшего к нему офиса. Ответ в этом задании — широта и долгота этого центра, записанные через пробел.

In [15]:
office_centers = np.array([
    [33.751277, -118.188740], [25.867736, -80.324116], 
    [51.503016, -0.075479], [52.378894, 4.885084], 
    [39.366487, 117.036146], [-33.868457, 151.205134]
])

In [16]:
distanses = []
for oc in office_centers:
    for c in centers15:
        d = np.sqrt((c[0] - oc[0])**2 + (c[1] - oc[1])**2)
        distanses.append((d, c))
distanses.sort()
distanses[:20]        

[(0.007834758163107856, array([-33.86063043, 151.20477593])),
 (0.009353316185992226, array([52.37296399,  4.89231722])),
 (0.022674066158385495, array([ 25.84567226, -80.3188906 ])),
 (0.05005829482278787, array([51.50299126, -0.12553729])),
 (0.07084773242719973, array([  33.80987796, -118.14892381])),
 (0.13410903336184654, array([ 25.78581242, -80.21793804])),
 (0.16740596425035326, array([ 25.70534972, -80.28342874])),
 (0.18887596060185083, array([ 26.01009825, -80.19999059])),
 (0.19577945647763628, array([  33.88832534, -118.04892817])),
 (0.21181053682436798, array([  33.87298601, -118.36209115])),
 (0.2222332907317907, array([  33.97257482, -118.16837067])),
 (0.2713007595066735, array([ 26.13884379, -80.33434684])),
 (0.2949788868004569, array([  33.98393587, -118.00740497])),
 (0.3022701186924605, array([ 26.12086266, -80.15890668])),
 (0.30473050307840693, array([  33.81730643, -117.89124917])),
 (0.3148837903362732, array([  34.06039755, -118.24870903])),
 (0.338810470251

In [17]:
with open('result1.txt', 'w') as fout:
    fout.write('{0} {1}'.format(distanses[0][1][0], distanses[0][1][1]))